In [4]:
import paddle
from paddlenlp.embeddings import TokenEmbedding, list_embedding_name
paddle.set_device("cpu")

print(list_embedding_name()) # ['w2v.baidu_encyclopedia.target.word-word.dim300']
token_embedding = TokenEmbedding(embedding_name="w2v.baidu_encyclopedia.target.word-word.dim300")
print(token_embedding)

NameError: name 'type_check' is not defined

In [25]:
#text similarity

In [13]:
test_token_embedding = token_embedding.search("中国")
from collections import Counter

def get_order_dict_N(_dict, N):
    result = Counter(_dict).most_common(N)
    d = {}
    for k,v in result:
        d[k] = v
    return d

txt_list=['中国人','美国汽车','中国','中国是一个国家','你好']

dic={}
def search_list(input_txt,target_list,method='cosine',topk=5):
    score=[]
    for txt in target_list:
        dic[txt]=token_embedding.cosine_sim(input_txt, txt)  
    return get_order_dict_N(dic,topk)

print(search_list('中国',txt_list,method='cosine',topk=5))

{'中国': 1.0000001, '中国人': 0.50604516, '你好': 0.118777126, '美国汽车': 0.040412646, '中国是一个国家': 0.040412646}


In [ ]:
#key words extraction

In [2]:
import jieba.analyse

sent = '我今天感到很焦虑，焦虑的原因是我很焦虑'
print('TF-IDF')
t = jieba.analyse.TFIDF()
kws = t.extract_tags(sent, topK=10, withWeight=True)
for word, weight in kws:
    print('%s %.4f'%(word, weight))

print('textrank')
t = jieba.analyse.TextRank()
kws = t.extract_tags(sent, topK=10, withWeight=True, allowPOS=('ns', 'n', 'vn', 'v'))
for word, weight in kws:
    print('%s %.4f'%(word, weight))

TF-IDF
焦虑 4.1189
感到 0.8770
今天 0.8320
原因 0.8253
textrank


In [3]:
import paddlenlp
from paddlenlp.transformers import SkepForSequenceClassification, SkepTokenizer
import os
import paddle
from functools import partial

import numpy as np
import paddle.nn.functional as F
from paddlenlp.data import Stack, Tuple, Pad

def convert_example(example,
                    tokenizer,
                    max_seq_length=512,
                    is_test=False):
    encoded_inputs = tokenizer(
        text=example["text"], max_seq_len=max_seq_length)


    input_ids = encoded_inputs["input_ids"]
    # token_type_ids
    token_type_ids = encoded_inputs["token_type_ids"]

    if not is_test:
        # label
        label = np.array([example["label"]], dtype="int64")
        return input_ids, token_type_ids, label
    else:
        # qid = np.array([example["qid"]], dtype="int64")
        return input_ids, token_type_ids

def create_dataloader(dataset,
                      trans_fn=None,
                      mode='train',
                      batch_size=1,
                      batchify_fn=None):
    if trans_fn:
        dataset = dataset.map(trans_fn)

    shuffle = True if mode == 'train' else False
    if mode == "train":
        sampler = paddle.io.DistributedBatchSampler(
            dataset=dataset, batch_size=batch_size, shuffle=shuffle)
    else:
        sampler = paddle.io.BatchSampler(
            dataset=dataset, batch_size=batch_size, shuffle=shuffle)
    dataloader = paddle.io.DataLoader(
        dataset, batch_sampler=sampler, collate_fn=batchify_fn)
    return dataloader
def predict(model, data, tokenizer, label_map, batch_size=1):
    """
    Predicts the data labels.

    Args:
        model (obj:`paddle.nn.Layer`): A model to classify texts.
        data (obj:`List(Example)`): The processed data whose each element is a Example (numedtuple) object.
            A Example object contains `text`(word_ids) and `se_len`(sequence length).
        tokenizer(obj:`PretrainedTokenizer`): This tokenizer inherits from :class:`~paddlenlp.transformers.PretrainedTokenizer` 
            which contains most of the methods. Users should refer to the superclass for more information regarding methods.
        label_map(obj:`dict`): The label id (key) to label str (value) map.
        batch_size(obj:`int`, defaults to 1): The number of batch.

    Returns:
        results(obj:`dict`): All the predictions labels.
    """
    examples = []
    for text in data:
        input_ids, segment_ids = convert_example(
            text,
            tokenizer,
            max_seq_length=128,
            is_test=True)
        examples.append((input_ids, segment_ids))

    batchify_fn = lambda samples, fn=Tuple(
        Pad(axis=0, pad_val=tokenizer.pad_token_id),  # input id
        Pad(axis=0, pad_val=tokenizer.pad_token_id),  # segment id
    ): fn(samples)

    # Seperates data into some batches.
    batches = []
    one_batch = []
    for example in examples:
        one_batch.append(example)
        if len(one_batch) == batch_size:
            batches.append(one_batch)
            one_batch = []
    if one_batch:
        # The last batch whose size is less than the config batch_size setting.
        batches.append(one_batch)

    results = []
    model.eval()
    for batch in batches:
        input_ids, segment_ids = batchify_fn(batch)
        input_ids = paddle.to_tensor(input_ids)
        segment_ids = paddle.to_tensor(segment_ids)
        logits = model(input_ids, segment_ids)
        probs = F.softmax(logits, axis=1)
        idx = paddle.argmax(probs, axis=1).numpy()
        idx = idx.tolist()
        labels = [label_map[i] for i in idx]
        results.extend(labels)
    return results


model = SkepForSequenceClassification.from_pretrained(pretrained_model_name_or_path="skep_ernie_1.0_large_ch", num_classes=19)
tokenizer = SkepTokenizer.from_pretrained(pretrained_model_name_or_path="skep_ernie_1.0_large_ch")

batch_size=1
params_path = '../model_state.pdparams'
vocab_path='skep_ckpt/model_800/vocab.txt'

if params_path and os.path.isfile(params_path):
    # load
    state_dict = paddle.load(params_path)
    model.set_dict(state_dict)
    # tokenizer.load_vocabulary(vocab_path)
    print("Loaded parameters from %s" % params_path)

print('init done')
ch_txt="哎我晚上一直睡不着怎么办呀"
data=[{'text':str(ch_txt),'qid':''}]
# data = [
#     {"text":'我担心我和同学和舍友的关系','qid':''},
#     {"text":'我挺开心的没事 哈哈哈啊哈','qid':''},
#     {"text":'我女朋友对我太坏了，我被那个坏女人伤透了心','qid':''},
#     {"text":'太难了学不下去了','qid':''},
#      {"text":'哎我晚上一直睡不着怎么办呀','qid':''},
# ]

label_map = {'低自尊': 0, '其他': 1, '失眠': 2, '情感关系问题': 3, '青春期问题': 4, '压力': 5, '自我探索': 6, '家庭问题和矛盾': 7, '强迫症': 8, '人际关系': 9, '事业和工作烦恼': 10,
 '学业烦恼、对未来规划的迷茫': 11, '离婚': 12, '分手': 13, '物质滥用': 14, '悲恸': 15, '性问题': 16, 'LGBT': 17, '亲子关系': 18, '尚未到达S2': 19, '忧郁症': 20, 
 '焦虑症': 21, '其他疾病': 22, '躁郁症': 23, '创伤后应激反应': 24, '恐慌症': 25, '厌食症和暴食症': 26, '进行的人身伤害': 27, '策划及逆行的自杀行为': 28, '自残': 29, '无伤害身体倾向': 30, '计划的人身伤害': 31}
label_map = {value:key for key,value in label_map.items()}
# print(label_map)

results = predict(
    model, data, tokenizer, label_map, batch_size=batch_size)
# for idx, text in enumerate(data):
#     print('Data: {} \t Label: {}'.format(text, results[idx]))
ans=str(results[0])
if(ans=="其他"):
    ans="你看起来还行呀 没事的 明天会更好"
else:
    ans="你可能有点关于"+ans+"的问题,你可以查看下这个网站来获取更多信息 https://zh.wikihow.com/wikiHowTo?search="+ans
print(ans)

[2021-10-27 21:28:04,166] [    INFO] - Already cached /home/aistudio/.paddlenlp/models/skep_ernie_1.0_large_ch/skep_ernie_1.0_large_ch.pdparams
[2021-10-27 21:28:08,282] [    INFO] - Already cached /home/aistudio/.paddlenlp/models/skep_ernie_1.0_large_ch/skep_ernie_1.0_large_ch.vocab.txt


Loaded parameters from ../model_state.pdparams
init done
你可能有点关于失眠的问题,你可以查看下这个网站来获取更多信息 https://zh.wikihow.com/wikiHowTo?search=失眠
